In [1]:
import tensorflow as tf
import os
import glob

__ORIG_WD__ = os.getcwd()

os.chdir("../utils")

from utils import print_progress_bar
kmer_len = 16

2023-08-19 11:45:50.916672: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-19 11:45:50.942814: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 11:45:51.392084: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_reads(fastq_file):
    """Reads a fastq file and returns a list of reads."""
    reads = []
    with open(fastq_file) as f:
        while True:
            f.readline() # ignore the first line
            seq = f.readline().strip()
            f.readline() # ignore the third line
            qual = f.readline().strip()
            if len(seq) == 0:
                break
            reads.append(seq)
    return reads

In [3]:
key = tf.random.stateless_uniform([1], minval=0, maxval=4**kmer_len, dtype=tf.int64, seed=[0,0]).numpy()[0]

base_to_int = {'A': 1, 'C': 2, 'G': 3, 'T': 4}

def get_signed_frag_value(frag, kmer_len):
    kmer = 0
    for i in range(kmer_len):
        kmer += base_to_int.get(frag[i], 0) * 5**i

    # perform bitwise XOR with key
    kmer ^= key
    return kmer

    
def get_frags(fastq_file, kmer_len, frag_len, num_frags):
    
    reads = get_reads(fastq_file)
    frags = []
    for read_idx, read in enumerate(reads):
        for frag_idx in range(len(read) - frag_len + 1):
            frags.append(((read_idx, frag_idx), get_signed_frag_value(read[frag_idx:frag_idx + frag_len], kmer_len)))
    # sort and filter out same values
    frags.sort(key=lambda x: x[1], reverse=True)
    frags = frags[0:0] + [frags[i] for i in range(1, len(frags)) if frags[i][1] != frags[i-1][1]]
    return [reads[read_idx][frag_idx:frag_idx + frag_len] for (read_idx, frag_idx), _ in frags[:num_frags]]


2023-08-19 11:45:51.927727: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 11:45:51.942811: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 11:45:51.942935: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
frags = get_frags(f"../genome_builder/data/reads-8x/BS001393.fq", 16, 86, 400)

In [5]:
tf.strings.bytes_split(tf.constant(frags)).to_tensor()

<tf.Tensor: shape=(400, 86), dtype=string, numpy=
array([[b'G', b'T', b'T', ..., b'A', b'T', b'A'],
       [b'G', b'T', b'T', ..., b'C', b'T', b'G'],
       [b'A', b'C', b'C', ..., b'T', b'A', b'T'],
       ...,
       [b'A', b'G', b'T', ..., b'A', b'A', b'T'],
       [b'A', b'G', b'C', ..., b'A', b'T', b'T'],
       [b'T', b'C', b'T', ..., b'T', b'T', b'C']], dtype=object)>

In [6]:
def frags_generator(dirpath: str, kmer_len, frag_len, num_frags) -> tf.Tensor:
    # extract fastq files from dirpath
    reads_file_pathes = glob.glob(os.path.join(dirpath, '*.fq')) + glob.glob(os.path.join(dirpath, '*.fastq'))
    for reads_file_path in reads_file_pathes:
        frags = get_frags(reads_file_path, kmer_len, frag_len, num_frags)
        print(frags)
        yield frags

In [7]:
# # print progress bar

# cnt = 0
# everything = len(glob.glob("../genome_builder/data/reads-0.5x/*.fq"))
# for frags in frags_generator("../genome_builder/data/reads-0.5x/", 16, 86, 400):
#     print_progress_bar(cnt, everything)
#     cnt += 1

In [8]:
dataset = tf.data.Dataset.from_generator(
    lambda: frags_generator("../genome_builder/data/reads-0.5x/", 16, 86, 400),
    output_signature=(
        tf.TensorSpec(shape=(0))
    )
)

In [9]:
for elem in dataset:
    print(elem)
    break

[]
tf.Tensor([], shape=(0,), dtype=float32)
[]


2023-08-19 11:45:52.952333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [12]:
import os
import glob

# change names of models:
for dirpath in glob.glob(f"data/covid19-1024examples/ml_models/*"):
    _, encoder_repeats, coverage =dirpath.split("/")[-1].split("-")[0].split(".")
    coverage = coverage[:-1]
    second_dirpath = glob.glob(f"{dirpath}/*")[0]
    print(coverage, encoder_repeats)
    
    # if coverage = 5:
    # # change names of second directory
    # os.system(f"mv {second_dirpath} {dirpath}/vit.{encoder_repeats}.{coverage}x")

    # # change names of first directory
    # os.system(f"mv {dirpath} {'/'.join(dirpath.split('/')[:-1])}/vit.{encoder_repeats}.{coverage}x-vit")
    
    # new_dirpath_name = 

2x 2
1x 2
0x 2
2x 3
5x 1
5x 3
5x 4
5x 2
1x 1
0x 3


In [ ]:
import tensorflow as tf

fastq_data = tf.constant("""@ENA|BS001393|BS001393.1-99
CCAAATATTTAATTGGTGGTGTTTTG
+
C1C1GGGGGGGGGGJJJJJJJJJJJGJ1JJJJJGJJJGGCGJJJ(GCGGCJJJJCGJJGGGGGJGGGGGGGJGCGGGGGGGGGGGG8GGGCGGGGGCCGGJCGCGGGCGGGCCCGGGGGCGGGCGGGGGGGGCGGGCGGGGG11GGGGGC
@ENA|BS001393|BS001393.1-98
AAAGTGATGTCAATGTCACTAACAAG
+
C=CGGG=GGGGGCJCJJGJGJCJJJJGGJGGJGJGJJGGJJGJJGJGJJJCG8JCGGGJCG=JJJCGGGGGGGCCC1G=GGCGGGGCGGCGGCGGCGGGC=CCGGGCGGCG8CGGCGGGGCGGCGGGG8CGCG===GCGCG=CGGG=GGG
@ENA|BS001393|BS001393.1-97
AATTCCATTGTTTGTAGATTTGACAC
+""")